# [Exporting 🤗 Transformers Models](https://huggingface.co/docs/transformers/serialization)

## Using TorchScript in Python

Below is an example, showing how to save, load models as well as how to use the trace for inference. 

### Saving a model
This snippet shows how to use TorchScript to export a BertModel. Here the BertModel is instantiated according to a BertConfig class and then saved to disk under the filename traced_bert.pt

In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor
import audio2numpy as a2n
import numpy as np
import torch
import torchaudio


In [2]:
def load_mp3_to_array_fn(path: str):
    r"""replace: torchaudio.load(path) which cannot load .mp3"""
    speech_array, _ = a2n.audio_from_file(path)
    speech_array = torch.tensor(np.array([speech_array]))
    resampler = torchaudio.transforms.Resample(48_000, 16_000)
    tensor = resampler(speech_array).squeeze().numpy()
    return tensor


In [3]:
# Instantiating the model
model = Wav2Vec2ForCTC.from_pretrained(
    "airesearch/wav2vec2-large-xlsr-53-th").to("cuda")


C:\Users\otash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\configuration_utils.py:353: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
# The model needs to be in evaluation mode
model.eval()


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), 

In [5]:
# Instantiating the processor
processor = Wav2Vec2Processor.from_pretrained(
    "airesearch/wav2vec2-large-xlsr-53-th")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Test original model from huggingface
path = r"./mp3/common_voice_th_23646618.mp3"

tensor = load_mp3_to_array_fn(path)

inputs = processor(tensor, sampling_rate=16_000,
                   return_tensors="pt", padding=True)

input_values = inputs.input_values.to("cuda")


In [7]:
# Creating the trace
traced_model = torch.jit.trace(model, input_values, strict=False)
torch.jit.save(traced_model, "./model/airesearch-wav2vec2-large-xlsr-53-th.pt")


C:\Users\otash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:622: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
C:\Users\otash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:659: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output

### Loading a model
This snippet shows how to load the BertModel that was previously saved to disk under the name traced_bert.pt. We are re-using the previously initialised dummy_input.

In [8]:
loaded_model = torch.jit.load(
    "./model/airesearch-wav2vec2-large-xlsr-53-th.pt")
loaded_model.eval()


RecursiveScriptModule(
  original_name=Wav2Vec2ForCTC
  (wav2vec2): RecursiveScriptModule(
    original_name=Wav2Vec2Model
    (feature_extractor): RecursiveScriptModule(
      original_name=Wav2Vec2FeatureEncoder
      (conv_layers): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=Wav2Vec2LayerNormConvLayer
          (conv): RecursiveScriptModule(original_name=Conv1d)
          (layer_norm): RecursiveScriptModule(original_name=LayerNorm)
        )
        (1): RecursiveScriptModule(
          original_name=Wav2Vec2LayerNormConvLayer
          (conv): RecursiveScriptModule(original_name=Conv1d)
          (layer_norm): RecursiveScriptModule(original_name=LayerNorm)
        )
        (2): RecursiveScriptModule(
          original_name=Wav2Vec2LayerNormConvLayer
          (conv): RecursiveScriptModule(original_name=Conv1d)
          (layer_norm): RecursiveScriptModule(original_name=LayerNorm)
        )
        (3): Recurs

### Testing the model

In [9]:
model(input_values)


CausalLMOutput(loss=None, logits=tensor([[[-12.7827, -19.4572,  -5.3144,  ...,  -4.5371, -21.6783,  21.0082],
         [-12.9260, -19.5551,  -5.2711,  ...,  -4.4008, -21.7991,  21.0341],
         [-13.0499, -19.6765,  -5.2720,  ...,  -4.4381, -22.0397,  21.0938],
         ...,
         [-13.9024, -20.0628,  -6.0033,  ...,  -6.3624, -23.8290,  21.6773],
         [-14.2578, -19.8381,  -5.7301,  ...,  -6.1568, -23.7448,  21.8396],
         [-13.2361, -19.5109,  -5.5667,  ...,  -5.4785, -22.3863,  21.2749]]],
       device='cuda:0', grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

In [10]:
loaded_model(input_values)


{'logits': tensor([[[-12.7827, -19.4572,  -5.3144,  ...,  -4.5371, -21.6783,  21.0082],
          [-12.9260, -19.5551,  -5.2711,  ...,  -4.4008, -21.7991,  21.0341],
          [-13.0499, -19.6765,  -5.2720,  ...,  -4.4381, -22.0397,  21.0938],
          ...,
          [-13.9024, -20.0628,  -6.0033,  ...,  -6.3624, -23.8290,  21.6773],
          [-14.2578, -19.8381,  -5.7301,  ...,  -6.1568, -23.7448,  21.8396],
          [-13.2361, -19.5109,  -5.5667,  ...,  -5.4785, -22.3863,  21.2749]]],
        device='cuda:0', grad_fn=<AddBackward0>)}

In [10]:
with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))


Prediction: ['พวกเรา อาจจะ ต้องการ ความช่วยเหลือ จาก เธอ ในที่สุด']


In [11]:
with torch.no_grad():
    logits = loaded_model(input_values)['logits']

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))


Prediction: ['พวกเรา อาจจะ ต้องการ ความช่วยเหลือ จาก เธอ ในที่สุด']
